##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Modelleri kaydedelim ve tekrar yükleyelim

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_restore_models"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Eğitim sırasında veya sonrasında modelin ilerleyişi kaydedilebilir. Bunun anlamı, modelin kaldığı yerden ilerlemeye devam edebilir olması, ayrıca uzun eğitim sürelerinin önüne geçilebilir olmasıdır. Modellerin kaydedilmesi, aynı zamanda modellerin paylaşılabilmesi ve diğerlerinin yaptığımız çalışmaları tekrardan oluşturabilmeleri anlamına gelmektedir. Çoğu uygulayıcı araştırma modellerini ve tekniklerini yayınladıklarında, aşağıdaki bilgileri paylaşırlar: 

* modeli oluşturan kodu, ve
* modele ait eğitilmiş ağırlık değerlerini, veya parametrelerini

Bu verilerin paylaşılması, diğerlerinin modelimizin nasıl çalıştığını anlamasına ve yeni veriler ile modeli denemelerine yardımcı olur. 

Dikkat: Güvenilmeyen kodlar ile ilgili dikkatli olunuz-Tensorflow modelleri kodlardan oluşmaktadır. Detaylar için [TensorFlow'un Güvenli Kullanımı](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) linkine göz atınız.

### Seçenekler

Kullandığınız API'ye bağlı olarak, Tensorflow modellerini kaydetmenin farklı yolları vardır. Bu eğitim dökümanı, Tensorflowda yapay zeka modellerinin oluşturulması ve eğitilmesinde kullanılan [tf.keras](https://www.tensorflow.org/guide/keras) 'ı kullanmaktadır. Farklı yöntemler için TensorFlow [Kaydet ve Geri Yükle](https://www.tensorflow.org/guide/saved_model) eğitim dökümanına veya [eager'da kaydedelim](https://www.tensorflow.org/guide/eager#object-based_saving) göz atailirsiniz.


## Setup

### Kuralım ve İçeri Alalım

Tensorflow ve bağlı kütüphanelerini kuralım ve içeri alalım:

In [0]:
!pip install h5py pyyaml 

### Örnek veri setini alalım

Ağırlıkların nasıl kaydedildiğini gösterebilmek için, modelimizi [MNIST dataset](http://yann.lecun.com/exdb/mnist/) verisi ile eğiteceğiz. Gözterimi hızlandırmak için, sadece ilk 1000 örneği kullanacağız:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Modeli tanımlayalım

Ağırlıkların nasıl kaydedileceğini ve yükleneceğini gösterebilmek için basit bir model oluşturalım.

In [0]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

## Eğitim sırasında kontrol noktalarını (checkpoints) kaydedelim

Ana kullanım şekli, eğitim sırasında ve sonunda kontrol noktalarının otomatik olarak kaydedilmesidir. Bu şekilde eğitilmiş modeli tekrar eğitmeye gerek kalmadan kullanabiliriz veya eğitim süreci yarıda kalmışsa kaldığı yerden eğitime devam edebiliriz. 

`tf.keras.callbacks.ModelCheckpoint` bu işlemi yapan callback fonksiyonudur. Bu fonksiyon, kontrol noktalarını yapılandırmak için birkaç parametre değeri alır. 

### Kontrol noktası callbak fonksiyonu kullanımı

Modeli eğitelim ve `ModelCheckpoint` callback fonksiyonunu modele aktaralım:

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Bu kod, her bir ephoc sonunda güncellenen, bir grup TensorFlow kontrol noktası dosyaları oluşturur:

In [0]:
!ls {checkpoint_dir}

Eğitilmemiş yeni bir model oluşturalım. Sadece ağırlık değerleri ile bir modeli geri yüklemek istediğimizde, elimizde orjinal model ile aynı yapıyı sahip bir modelimiz olmalıdır. Aynı model yapısına sahip olduğumuz için, farklı zamanlarda oluşmuş model örnekleri arasında ağırlık değerlerini paylaşabiliriz. 

Şimdi yeni bir eğitilmemiş model oluşturalım ve bu modeli test veri seti ile değerlendirelim. Eğitilmemiş bir model, şans yüzdesi kadar (~10% doğruluk) doğruluğa sahip olacaktır:

In [0]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Sonrasında ağırlık değerlerini kaydettiğimiz kontrol noktasından model geri yükleyelim ve modeli tekrardan değerlendirelim: 

In [0]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Kontrol noktası callback opsiyonları:

Callback fonksiyonu, kontrol noktalarının isimlendirilmesi ve frekanslarının ayarlanması için çeşitli seçenekler sunar. 

Yeni bir modeli, eğitelim ve her 5 epoch'ta bir farklı isimler ile kontrol noktalarını isimlendirelim:


In [0]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

Şimdi, oluşan kontrol noktalarına bakalım ve en güncel olanını seçelim:

In [0]:
! ls {checkpoint_dir}

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Not: Tensorflow varsayılan formatı, sadece en güncel 5 kontrol noktasını kaydeder.

Test için, modeli resetleyelim ve en güncel kontrol noktasını yükleyelim:

In [0]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Bu dosyalar nedir?

Yukardaki kod, ağırlık değerlerini bir grup [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)- dosyaya binary formatta kaydeder. Kontrol noktası aşağıdakiler kapsar: 
* Modele ait ağırlık değerlerini içeren, bir veya daha fazla dosya parçası (shards). 
* Hangi ağırlık değerinin hangi dosya parçasında olduğunu gösteren bir index dosyası. 

Eğer modelinizi tek bir bilgisayarda eğitiyorsanız, son takısı `.data-00000-of-00001` olan tek bir dosya parçası oluşacaktır.

## Ağırlıkların manuel kaydedilmesi

Yukarıda, ağırlıkların modele nasıl yüklendiğini gördünüz.

`Model.save_weights` methodunu kullanarak, ağırlıkların manuel olarak kaydedilmeside aynı şeklide kolaydır.

In [0]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Tüm modelin kaydedilmesi

Ağırlık değerleri, model yapısı ve hatta optimize edici parametrelerini (yapılandırmaya bağlı olarak) kapsayan tek bir dosya ile tüm model kaydedilebilir. Bu, orjinal koda ulaşmaya gerek kalmadan, modele ait kontrol noktasını kaydetmeyi ve sonrasında eğitime kalındığı yerden tekrardan başlanmasını sağlar. 

Tam-fonksiyonel modelin kaydedilmesi çok faydalıdır, bu modeli TensorFlow.js ye ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) yükleyip sonrasında eğitebilir ve web browserda çalıştırabiliriz. Veya modeli TensorFlow Lite formatına dönüştürerek ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_)) mobil cihazlarda çalıştırabiliriz

### Modeli HDF5 dosyası olarak kaydedelim

Keras, [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standartını kullanarak, temel bir kayıt formatı sunar. Bizim kullanım amacımıza göre, kayıtlı model tek bir binary blob olarak değerlendirilebilir. 

In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Şimdi, bu dosya ile modelimizi tekrardan oluşturalım:

In [0]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Doğruluğunu kontrol edelim:

In [0]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Bu yöntem modelle ilgili herşeyi kaydeder:

* Ağırlık değerlerini
* Model yapısını
* Optimizer parametrelerini

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.


### As a `saved_model`

Caution: This method of saving a `tf.keras` model is experimental and may change in future versions. 

Build a fresh model:

In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Create a `saved_model`: 

In [0]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")

Saved models are placed in a time-stamped directory:

In [0]:
!ls saved_models/

Reload a fresh keras model from the saved model.

In [0]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

Run the restored model.

In [0]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## What's Next

That was a quick guide to saving and loading in with `tf.keras`.

* The [tf.keras guide](https://www.tensorflow.org/guide/keras) shows more about saving and loading models with `tf.keras`.

* See [Saving in eager](https://www.tensorflow.org/guide/eager#object_based_saving) for saving during eager execution.

* The [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide has low-level details about TensorFlow saving.